# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

I've downloaded the dataset from [here](https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip)

In [ ]:
# Load pickled data
import pickle

print('Loading pickled data...')

training_file = 'data/train.p'
validation_file = 'data/valid.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

assert(len(X_train) == len(y_train))
assert(len(X_valid) == len(y_valid))
assert(len(X_test) == len(y_test))

print('Loaded')

---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Provide a Basic Summary of the Data Set Using Python, Numpy and/or Pandas

In [ ]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

import numpy as np

# Number of training examples
n_train = len(X_train)

# Number of validation examples
n_validation = len(X_valid)

# Number of testing examples.
n_test = len(X_test)

# What's the shape of an traffic sign image?
image_shape = X_train[0].shape

# How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of validation examples =", n_validation)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

### Include an exploratory visualization of the dataset

Visualize the German Traffic Signs Dataset using the pickled file(s). This is open ended, suggestions include: plotting traffic sign images, plotting the count of each sign, etc. 

The [Matplotlib](http://matplotlib.org/) [examples](http://matplotlib.org/examples/index.html) and [gallery](http://matplotlib.org/gallery.html) pages are a great resource for doing visualizations in Python.

**NOTE:** It's recommended you start with something simple first. If you wish to do more, come back to it after you've completed the rest of the sections. It can be interesting to look at the distribution of classes in the training, validation and test set. Is the distribution the same? Are there more examples of some classes than others?

In [ ]:
### Data exploration visualization code goes here.
### Feel free to use as many code cells as needed.
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

def drawLabelsDistibution(data, title):
    fig, (ax) = plt.subplots(ncols=1, figsize=(20, 5))
    labels, features_per_label = np.unique(data, return_counts=True)
    ax.bar(labels, features_per_label)
    ax.set_xticks(labels)
    ax.set_ylabel('Number of samples')
    ax.set_title('Signs distribution: %s' % title)
    fig.tight_layout()
    plt.show()

In [ ]:
# Distribution of classes in the training set
drawLabelsDistibution(y_train, 'training')

In [ ]:
# Distribution of classes in the validation set
drawLabelsDistibution(y_valid, 'validation')

In [ ]:
# Distribution of classes in the test set
drawLabelsDistibution(y_test, 'test')

In [ ]:
def draw_image_grid(images, columns=3, fname=None):
    """
    A helper method for drawing multiple images in a grid.
    Takes in a list of images to draw.
    """
    rows = len(images) / columns + 1
    row_height = 12 / columns
    f = plt.figure(figsize=(20, row_height*rows)) # width, height in inches. If not provided, defaults to rc figure.figsize.
    for i, img in enumerate(images):
        splt = plt.subplot(rows, columns, i + 1)
        if len(img.shape) > 2:
            splt.imshow(img)
        else:
            splt.imshow(img, cmap='gray') 
    plt.tight_layout()        
    if fname is not None:
        plt.savefig(fname, bbox_inches='tight')
        
def draw_annotated_image_grid(annotated_images, columns=3, fname=None):
    """
    A helper method for drawing multiple images in a grid.
    Takes in a list of pairs (title, image) to draw.
    """
    rows = len(annotated_images) / columns + 1
    row_height = 12 / columns
    f = plt.figure(figsize=(20, row_height*rows)) # width, height in inches. If not provided, defaults to rc figure.figsize.
    for i, annotated_image in enumerate(annotated_images):
        title = annotated_image[0]
        image = annotated_image[1]
        splt = plt.subplot(rows, columns, i + 1)
        splt.set_title(title, fontsize=20)
        if len(image.shape) > 2:
            splt.imshow(image)
        else:
            splt.imshow(image, cmap='gray')
    plt.tight_layout()                    
    if fname is not None:
        plt.savefig(fname, bbox_inches='tight')        

In [ ]:
## Shows an example of each class

labels, indices = np.unique(y_train, return_index=True)
print(labels)
print(indices)

imgs = []
for i in range(len(labels)):
    label = labels[i]
    idx = indices[i]
    img = X_train[idx]
    imgs.append(("%2d" % label, img))
    
draw_annotated_image_grid(imgs, columns=8)

----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Adding jittering

In [ ]:
import random

# Randomly translates image [-2..2] px
def r_translate(img):
    rows,cols = img.shape[:2]
    dx = random.randint(-2,2)
    dy = random.randint(-2,2)
    M = np.float32([[1,0,dx],[0,1,dy]])
    return cv2.warpAffine(img,M,(cols,rows))

index = random.randint(0, len(X_train))
image = X_train[index]
image_tr = r_translate(image)

plt.figure(figsize=(1,1))
plt.imshow(image.squeeze())
plt.figure(figsize=(1,1))
plt.imshow(image_tr.squeeze())

In [ ]:
import random

# Randomly rotates image [-15..15] degrees
def r_rotate(img):
    rows,cols = img.shape[:2]
    angle = random.randint(-15,15)
    M = cv2.getRotationMatrix2D((cols/2,rows/2),angle,1)
    return cv2.warpAffine(img,M,(cols,rows))    

index = random.randint(0, len(X_train))
image = X_train[index]
image_tr = r_rotate(image)

plt.figure(figsize=(1,1))
plt.imshow(image.squeeze())
plt.figure(figsize=(1,1))
plt.imshow(image_tr.squeeze())

In [ ]:
# Creates jittered copy of image
def jitter(img):
    img = r_translate(img)
    return r_rotate(img)

index = random.randint(0, len(X_train))
image = X_train[index]
image_tr = jitter(image)

plt.figure(figsize=(1,1))
plt.imshow(image.squeeze())
plt.figure(figsize=(1,1))
plt.imshow(image_tr.squeeze())

In [ ]:
labels = np.unique(y_train)
print(labels)

target_cnt = 2500

X_extended = []
y_extended = []

for label in labels:
    X = X_train[y_train == label]
    n_examples = len(X)
    n_jittered = target_cnt - n_examples
    print('Adding %d image to class %d' % (n_jittered, label))
    X_jittered = []
    for i in range(n_jittered):
        r_idx = random.randint(0, n_examples-1)
        img = jitter(X[r_idx])
        X_jittered.append(img)
    X = np.concatenate((X, X_jittered))
    assert(len(X) == target_cnt)
    X_extended.append(X)
    y_extended.append(np.repeat(label, target_cnt))
    
X_extended = np.concatenate(X_extended)
y_extended = np.concatenate(y_extended)

In [ ]:
# Distribution of classes in the extended training set
drawLabelsDistibution(y_extended, 'extended training')

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

In [ ]:
import random

# To Grayscale
def rgb2gray(rgb):
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    return np.expand_dims(gray, axis=len(gray.shape))

index = random.randint(0, len(X_train))
image = X_train[index]
image_gray = rgb2gray(image)

print("Image shape: ", image.shape)
print("Gray image shape: ", image_gray.shape)

plt.figure(figsize=(1,1))
plt.imshow(image.squeeze())
plt.figure(figsize=(1,1))
plt.imshow(image_gray.squeeze(), cmap="gray")

In [ ]:
# Normalization
def norm(image):
    return (image - 128.) / 128.

index = random.randint(0, len(X_train))
feature = X_train[index]
feature_norm = norm(feature)

print(feature[0][0])
print(feature_norm[0][0])

In [ ]:
def preprocessing_pipiline(X):
    X_gray = np.asarray([rgb2gray(x) for x in X])
    X_norm = norm(X_gray)
    return X_norm

print(type(X_extended))
print(X_extended.shape)

X_train_norm = preprocessing_pipiline(X_extended)
X_valid_norm = preprocessing_pipiline(X_valid)
X_test_norm = preprocessing_pipiline(X_test)
    
print('Data preprocessed')

### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

import tensorflow as tf

def conv2d(x, W, b, strides):
    conv = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding="VALID")
    conv = tf.nn.bias_add(conv, b)
    return tf.nn.relu(conv)

def conv2d_lrelu(x, W, b, strides):
    conv = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding="VALID")
    conv = tf.nn.bias_add(conv, b)
    return tf.nn.leaky_relu(conv)
    
def maxpool2d(x, k):
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1], padding="VALID")    

In [ ]:
def LeNet(x):
    """
    32x32x1 images => 43 traffic sign classes
    """
    
    mu = 0
    sigma = 0.1

    weights = {
        'l1': tf.Variable(tf.truncated_normal([5, 5, 1, 6], mean=mu, stddev=sigma)),
        'l2': tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=mu, stddev=sigma)),
        'l3': tf.Variable(tf.truncated_normal([400, 120], mean=mu, stddev=sigma)),
        'l4': tf.Variable(tf.truncated_normal([120, 84], mean=mu, stddev=sigma)),    
        'l5': tf.Variable(tf.truncated_normal([84, 43], mean=mu, stddev=sigma))    
    }

    biases = {
        'l1': tf.Variable(tf.zeros(6)),
        'l2': tf.Variable(tf.zeros(16)),
        'l3': tf.Variable(tf.zeros(120)),
        'l4': tf.Variable(tf.zeros(84)),
        'l5': tf.Variable(tf.zeros(43))    
    }    
    
    # Layer 1: 
    # - Convolutional. Input = 32x32x1. Output = 28x28x6.
    # - Activation.
    # - Pooling. Input = 28x28x6. Output = 14x14x6.
    l1_conv = conv2d(x, weights['l1'], biases['l1'], strides=1)
    l1_out = maxpool2d(l1_conv, k=2)

    ### -------------------------
    
    # Layer 2: 
    # - Convolutional. Output = 10x10x16.
    # - Activation.
    # - Pooling. Input = 10x10x16. Output = 5x5x16.
    l2_conv = conv2d(l1_out, weights['l2'], biases['l2'], strides=1)
    l2_out = maxpool2d(l2_conv, k=2)
    
    ### -------------------------    

    # Flatten. Input = 5x5x16. Output = 400.
    l3_input = tf.reshape(l2_out, [-1, 400])
    
    ### -------------------------    
    
    # Layer 3: 
    # - Fully Connected. Input = 400. Output = 120.
    # - Activation.
    l3_out = tf.add(tf.matmul(l3_input, weights['l3']), biases['l3'])
    l3_out = tf.nn.relu(l3_out)
    
    ### -------------------------    

    # Layer 4: 
    # - Fully Connected. Input = 120. Output = 84.
    # - Activation.
    l4_out = tf.add(tf.matmul(l3_out, weights['l4']), biases['l4'])
    l4_out = tf.nn.relu(l4_out)
    
    
    ### -------------------------    
    
    # Layer 5: 
    # - Fully Connected. Input = 84. Output = 43.
    logits = tf.add(tf.matmul(l4_out, weights['l5']), biases['l5'])    
    
    return logits

In [ ]:
def LeNetDropout(x, dropout):
    """
    32x32x1 images => 43 traffic sign classes
    dropout <- (0..1)
    """
    
    mu = 0
    sigma = 0.1

    weights = {
        'l1': tf.Variable(tf.truncated_normal([5, 5, 1, 6], mean=mu, stddev=sigma)),
        'l2': tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=mu, stddev=sigma)),
        'l3': tf.Variable(tf.truncated_normal([400, 120], mean=mu, stddev=sigma)),
        'l4': tf.Variable(tf.truncated_normal([120, 84], mean=mu, stddev=sigma)),    
        'l5': tf.Variable(tf.truncated_normal([84, 43], mean=mu, stddev=sigma))    
    }

    biases = {
        'l1': tf.Variable(tf.zeros(6)),
        'l2': tf.Variable(tf.zeros(16)),
        'l3': tf.Variable(tf.zeros(120)),
        'l4': tf.Variable(tf.zeros(84)),
        'l5': tf.Variable(tf.zeros(43))    
    }    
    
    # Layer 1: 
    # - Convolutional. Input = 32x32x1. Output = 28x28x6.
    # - Activation.
    # - Pooling. Input = 28x28x6. Output = 14x14x6.
    l1_conv = conv2d(x, weights['l1'], biases['l1'], strides=1)
    l1_out = maxpool2d(l1_conv, k=2)

    ### -------------------------
    
    # Layer 2: 
    # - Convolutional. Output = 10x10x16.
    # - Activation.
    # - Pooling. Input = 10x10x16. Output = 5x5x16.
    l2_conv = conv2d(l1_out, weights['l2'], biases['l2'], strides=1)
    l2_out = maxpool2d(l2_conv, k=2)
    
    ### -------------------------    

    # Flatten. Input = 5x5x16. Output = 400.
    l3_input = tf.reshape(l2_out, [-1, 400])
    
    ### -------------------------    
    
    # Layer 3: 
    # - Fully Connected. Input = 400. Output = 120.
    # - Activation.
    l3_out = tf.add(tf.matmul(l3_input, weights['l3']), biases['l3'])
    l3_out = tf.nn.relu(l3_out)
    l3_out = tf.nn.dropout(l3_out, keep_prob=1.0-dropout)
    
    ### -------------------------    

    # Layer 4: 
    # - Fully Connected. Input = 120. Output = 84.
    # - Activation.
    l4_out = tf.add(tf.matmul(l3_out, weights['l4']), biases['l4'])
    l4_out = tf.nn.relu(l4_out)
    l4_out = tf.nn.dropout(l4_out, keep_prob=1.0-dropout)
    
    ### -------------------------    
    
    # Layer 5: 
    # - Fully Connected. Input = 84. Output = 43.
    logits = tf.add(tf.matmul(l4_out, weights['l5']), biases['l5'])    
    
    return logits

In [ ]:
def LeNetDropoutLRelu(x, dropout):
    """
    32x32x1 images => 43 traffic sign classes
    dropout <- (0..1)
    """
    
    mu = 0
    sigma = 0.1

    weights = {
        'l1': tf.Variable(tf.truncated_normal([5, 5, 1, 6], mean=mu, stddev=sigma)),
        'l2': tf.Variable(tf.truncated_normal([5, 5, 6, 16], mean=mu, stddev=sigma)),
        'l3': tf.Variable(tf.truncated_normal([400, 120], mean=mu, stddev=sigma)),
        'l4': tf.Variable(tf.truncated_normal([120, 84], mean=mu, stddev=sigma)),    
        'l5': tf.Variable(tf.truncated_normal([84, 43], mean=mu, stddev=sigma))    
    }

    biases = {
        'l1': tf.Variable(tf.zeros(6)),
        'l2': tf.Variable(tf.zeros(16)),
        'l3': tf.Variable(tf.zeros(120)),
        'l4': tf.Variable(tf.zeros(84)),
        'l5': tf.Variable(tf.zeros(43))    
    }    
    
    # Layer 1: 
    # - Convolutional. Input = 32x32x1. Output = 28x28x6.
    # - Activation.
    # - Pooling. Input = 28x28x6. Output = 14x14x6.
    l1_conv = conv2d_lrelu(x, weights['l1'], biases['l1'], strides=1)
    l1_out = maxpool2d(l1_conv, k=2)

    ### -------------------------
    
    # Layer 2: 
    # - Convolutional. Output = 10x10x16.
    # - Activation.
    # - Pooling. Input = 10x10x16. Output = 5x5x16.
    l2_conv = conv2d_lrelu(l1_out, weights['l2'], biases['l2'], strides=1)
    l2_out = maxpool2d(l2_conv, k=2)
    
    ### -------------------------    

    # Flatten. Input = 5x5x16. Output = 400.
    l3_input = tf.reshape(l2_out, [-1, 400])
    
    ### -------------------------    
    
    # Layer 3: 
    # - Fully Connected. Input = 400. Output = 120.
    # - Activation.
    l3_out = tf.add(tf.matmul(l3_input, weights['l3']), biases['l3'])
    l3_out = tf.nn.leaky_relu(l3_out)
    l3_out = tf.nn.dropout(l3_out, keep_prob=1.0-dropout)
    
    ### -------------------------    

    # Layer 4: 
    # - Fully Connected. Input = 120. Output = 84.
    # - Activation.
    l4_out = tf.add(tf.matmul(l3_out, weights['l4']), biases['l4'])
    l4_out = tf.nn.leaky_relu(l4_out)
    l4_out = tf.nn.dropout(l4_out, keep_prob=1.0-dropout)
    
    ### -------------------------    
    
    # Layer 5: 
    # - Fully Connected. Input = 84. Output = 43.
    logits = tf.add(tf.matmul(l4_out, weights['l5']), biases['l5'])    
    
    return logits

In [ ]:
def LeNetDropout25(x):
    return LeNetDropout(x, dropout=0.25)

def LeNetDropout50(x):
    return LeNetDropout(x, dropout=0.5)

def LeNetDropout50LRelu(x):
    return LeNetDropoutLRelu(x, dropout=0.5)

### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.

EPOCHS = 20
BATCH_SIZE = 64
rate = 0.001

x = tf.placeholder(tf.float32, (None, 32, 32, 1))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 43)


logits = LeNet(x)
#logits = LeNetDropout25(x)
#logits = LeNetDropout50(x)
#logits = LeNetDropout50LRelu(x)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

probabilities = tf.nn.softmax(logits)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
from sklearn.utils import shuffle

train_acc = []
valid_acc = []

# Training the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    features = X_train_norm
    labels = y_extended
    
    assert(len(features) == len(labels))
    
    num_examples = len(features)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        features, labels = shuffle(features, labels)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = features[offset:end], labels[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        training_accuracy = evaluate(X_train_norm, y_extended)
        validation_accuracy = evaluate(X_valid_norm, y_valid)
        
        train_acc.append(training_accuracy)
        valid_acc.append(validation_accuracy)
        
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))        
        print()
        
    saver.save(sess, './lenet')
    print("Model saved")

In [ ]:
x_axis = range(len(train_acc))

fig, (ax) = plt.subplots(ncols=1, figsize=(20, 5))
ax.set_xticks(x_axis)
ax.set_xlabel('Training Epoch')
ax.set_ylabel('Accuracy')
ax.set_title('Train-Validate Accuracy')

train_plot, = ax.plot(train_acc)
valid_plot, = ax.plot(valid_acc)

fig.tight_layout()
ax.legend([train_plot, valid_plot], ['Training Accuracy', 'Validation Accuracy'])
plt.show()

In [ ]:
# TODO: calculate test accuracy

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

import os
os.listdir("./traffic_signs")

In [ ]:
import os
import cv2
import matplotlib.image as mpimg

X_web = []
y_web = []
imgs = []
for filename in os.listdir("./traffic_signs"):
    fname = "./traffic_signs/" + filename
    img = mpimg.imread(fname)
    label = int(filename.split('_')[0])
    X_web.append(img)
    y_web.append(label)
    imgs.append((label, img))
    
X_web = np.asarray(X_web)
y_web = np.asarray(y_web)
    
print("Image shape: ", X_web[0].shape)
print("Labels: ", y_web)
    
draw_annotated_image_grid(imgs, columns=5)

### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

In [ ]:
X_web_norm = preprocessing_pipiline(X_web)

print("Web data prepared")

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    imgs = []
    probs = sess.run(probabilities, feed_dict={x: X_web_norm, y: y_web})
    for i in range(len(y_web)):
        ps = probs[i]
        label = y_web[i]
        p_idx = np.argmax(ps)
        print("Image %2d: label = %2d, prediction = %2d (%0.2f)" % (i, label, p_idx, ps[p_idx]))
        matched = 'OK' if label == p_idx else 'FAIL'
        title = "%s: %2d -> %2d (%0.2f)" % (matched, label, p_idx, ps[p_idx])
        imgs.append((title, X_web[i]))
        
draw_annotated_image_grid(imgs, columns=5)

### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    web_accuracy = evaluate(X_web_norm, y_web)
    print("Web Accuracy = {:.3f}".format(web_accuracy))

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")